In [1]:
# 上一節學習寫LR多分類的時候，網絡中的參數w和b都是自己手動定義的（而且w的shape是[輸出,輸入]

import torch
from torch import nn
from torch.nn import functional as F

# 模擬一張28x28的圖片攤平
x = torch.randn(1, 784)  # shape=[1,784]

# 定義三個全連接層
layer1 = nn.Linear(784, 200)  # 輸入784階輸出200階
layer2 = nn.Linear(200, 200)
layer3 = nn.Linear(200, 10)

x = layer1(x)  # shape=[1,200]
x = F.relu(x, inplace=True) # inplace=True在原對象基礎上修改,可以節省內存

x = layer2(x)  # shape=[1,200]
x = F.relu(x, inplace=True)

x = layer3(x)  # shape=[1,10]
x = F.relu(x, inplace=True)

# 自己定義網絡實現上節MNIST分類
# 更常用的用法是直接繼承 可以看到網絡中的參數不再暴露給用戶，這樣也不用去考慮參數初始化的問題了。 nn.Module

In [4]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms


class MLP(nn.Module):
    """自己定义网络的层,要继承nn.Module"""

    def __init__(self):
        """在构造器中定义层次结构"""
        super(MLP, self).__init__()
        # 在这里定义网络的每一层,可以添加任何继承nn.Module的类
        self.module = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 10),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        """定义前向过程"""
        x = self.module(x)
        return x


"""超参数"""
batch_size = 200  # 每批的样本数量
learning_rate = 0.01  # 学习率
epochs = 10  # 跑多少次样本集

"""获取训练集"""
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('../data', train=True,   # train=True则得到的是训练集
               transform=transforms.Compose([  # 进行数据预处理
                   transforms.ToTensor(),  # 这表示转成Tensor类型的数据
                   transforms.Normalize((0.1307,), (0.3081,))  # 这里是进行数据标准化(减去均值除以方差)
               ])),
batch_size=batch_size, shuffle=True)  # 按batch_size分出一个batch维度在最前面,shuffle=True打乱顺序

"""获取测试集"""
test_loader = torch.utils.data.DataLoader(
datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])),
batch_size=batch_size, shuffle=True)

"""训练+测试过程"""
net = MLP()
# 这里net.parameters()得到这个类所定义的网络的参数,各个w和各个b
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
Loss = nn.CrossEntropyLoss()

for epoch in range(epochs):
    """训练"""
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.reshape(-1, 28 * 28)  # 二维的图片数据摊平
        logits = net(data)  # 前面定义的网络MLP的输出
        loss = Loss(logits, target)  # nn.CrossEntropyLoss()自带Softmax
        optimizer.zero_grad()  # 梯度信息清空
        loss.backward()  # 反向传播获取梯度信息
        optimizer.step()  # 优化器执行
        # 每100个batch输出一次信息
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    """测试"""
    test_loss = 0  # 在测试集上的Loss,反映了模型的表现
    correct = 0  # 记录正确分类的样本数
    # 对测试集中每个batch的样本,标签
    for data, target in test_loader:
        # 摊平成shape=[样本数,784]的形状
        data = data.reshape(-1, 28 * 28)
        # 这里前向计算过程就是用定义的网络跑一下
        logits = net(data)
        test_loss += Loss(logits, target).item()
        # 得到的預測值輸出是一個10個分量的概率,在第2個維度上取max
        # logits.data是一個shape=[batch_size,10]的Tensor
        # 注意Tensor.max(dim=1)是在這個Tensor的1號維度上求最大值
        # 得到一個含有兩個元素的元組,這兩個元素都是shape=[batch_size]的Tensor
        # 第一個Tensor裡面存的都是最大值的值,第二個Tensor裡面存的是對應的索引
        # 這裡要取索引,所以取了這個tuple的第二個元素
        # print(type(logits.data), logits.data.shape,type(logits.data.max(dim=1)))
        pred = logits.data.max(dim=1)[1]
        # 对应位置相等则对应位置为True,这里用sum()即记录了True的数量
        correct += pred.eq(target.data).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


C:\Users\sam\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.292257
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.939222
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.420568

Test set: Average loss: 0.0046, Accuracy: 7684/10000 (77%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.933750
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.703012
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.642683

Test set: Average loss: 0.0023, Accuracy: 8810/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.533716
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.421376
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.364084

Test set: Average loss: 0.0017, Accuracy: 9020/10000 (90%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.255087
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.481018
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.249919

Test set: Average loss: 0.0015, Accuracy: 9102/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.311381
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.293084
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.273162

Test set: Average loss: 0.0014

# GPU加速

In [5]:
device = torch.device('cuda:0')
net=MLP().to(device)
Loss=nn.CrossEntropyLoss().to(device)
data,target=data.to(device),target.to(device)

AssertionError: Torch not compiled with CUDA enabled